# Drawing a treemap of the daily performance of securities listed in the Athens Stock Exchange General Index 
In this notebook we generate a treemap diagram for the stocks listed in the Athens Stock Exchange General Index

We use YAHOO! Finance to get the stock data and plotly express to draw the treemap diagram.

Example output:
<img src='https://i.imgur.com/BJbORP8.png'>

## Install / import libraries

In [1]:
# install libraries
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

from datetime import datetime

import warnings

# to generate the plotly diagram
import plotly.express as px
import plotly.graph_objects as go
#from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

## Generate dataset
*    Stock weight in the index is based on the [current](https://www.athexgroup.gr/web/guest/index-composition) index composition (as of 02 NOV 2022) 
*    Sector classification (second element in the list below) is based on [this](https://www.athexgroup.gr/documents/10180/5517704/ATHEX+Classification+Ver0.810+%28EN%29.pdf/5c49a27d-1078-4974-990b-5a5e1f278b73) ATHEX press release.

Prices are fetched using Yahoo Finance API's `tickerdata.info()` (`currentPrice` and `previousClose` attributes).


In [3]:
# list of ticker names composing the Athens Stock Exchange General Index (BBG: ASE:IND). The keys are the greek symbols and the values are a list with the yahoo finance ticker name, the industry and the index weight
ATGSTONKS = {
    'CENER':['CENER.AT','Industrial Goods & Services',0.57],
    'TITC':['TITC.AT','Construction & Materials',2.6243],
    'ΑΒΑΞ':['AVAX.AT','Construction & Materials',0.1954],
    'ΑΔΜΗΕ':['ADMIE.AT','Utilities',1.0591],
    'ΑΛΜΥ':['ALMY.AT','Basic Resources',0.071],
    'ΑΛΦΑ':['ALPHA.AT','Banks',5.2259],
    'ΑΡΑΙΓ':['AEGN.AT','Travel & Leisure',1.0148],
    'ΒΙΟ':['VIO.AT','Industrial Goods & Services',0.9212],
    'ΓΕΚΤΕΡΝΑ':['GEKTERNA.AT','Construction & Materials',3.335],
    'ΔΕΗ':['PPC.AT','Utilities',5.454],
    'ΕΕΕ':['EEE.AT','Food, Beverage & Tobacco',10.6946],
    'ΕΛΛΑΚΤΩΡ':['ELLAKTOR.AT','Construction & Materials',0.7655],
    'ΕΛΠΕ':['ELPE.AT','Energy',2.3321],
    'ΕΛΣΤΡ':['ELSTR.AT','Basic Resources',0.1519],
    'ΕΛΤΟΝ':['ELTON.AT','Industrial Goods & Services',0.0672],
    'ΕΛΧΑ':['ELHA.AT','Basic Resources',0.4894],
    'ΕΝΤΕΡ':['ENTER.AT','Technology',0.2747],
    'ΕΠΣΙΛ':['EPSIL.AT','Technology',0.5575],
    'ΕΤΕ':['ETE.AT','Banks',5.4313],
    'ΕΥΑΠΣ':['EYAPS.AT','Utilities',0.1433],
    'ΕΥΔΑΠ':['EYDAP.AT','Utilities',1.58],
    'ΕΥΡΩΒ':['EUROB.AT','Banks',5.2023],
    'ΕΧΑΕ':['EXAE.AT','Financial Services',1.0491],
    'ΙΑΤΡ':['IATR.AT','Health Care',0.1488],
    'ΙΚΤΙΝ':['IKTIN.AT','Construction & Materials',0.1073],
    'ΙΝΚΑΤ':['INKAT.AT','Construction & Materials',0.1254],
    'ΙΝΛΙΦ':['INLIF.AT','Insurance',0.1113],
    'ΙΝΤΕΚ':['INTEK.AT','Technology',0.2842],
    'ΙΝΤΕΡΚΟ':['INTERCO.AT','Real Estate',0.0826],
    'ΙΝΤΚΑ':['INTRK.AT','Telecommunications',0.5202],
    'ΚΑΜΠ':['KAMP.AT','Real Estate',0.2185],
    'ΚΕΚΡ':['KEKR.AT','Real Estate',0.0263],
    'ΚΟΥΕΣ':['QUEST.AT','Technology',0.6184],
    'ΚΡΙ':['KRI.AT','Food, Beverage & Tobacco',0.2584],
    'ΛΑΜΔΑ':['LAMDA.AT','Real Estate',2.8838],
    'ΛΟΥΛΗ':['KYLO.AT','Food, Beverage & Tobacco',0.0544],
    'ΜΙΓ':['MIG.AT','Financial Services',0.1069],
    'ΜΟΗ':['MOH.AT','Energy',4.8944],
    'ΜΟΤΟ':['MOTO.AT','Retail',0.1613],
    'ΜΠΕΛΑ':['BELA.AT','Consumer Products & Services',4.8227],
    'ΜΠΡΙΚ':['BRIQ.AT','Real Estate',0.1729],
    'ΜΥΤΙΛ':['MYTIL.AT','Industrial Goods & Services',5.2607],
    'ΟΛΘ':['OLTH.AT','Industrial Goods & Services',0.2759],
    'ΟΛΠ':['PPA.AT','Industrial Goods & Services',0.5774],
    'ΟΛΥΜΠ':['OLYMP.AT','Consumer Products & Services',0.1454],
    'ΟΠΑΠ':['OPAP.AT','Travel & Leisure',4.6455],
    'ΟΤΕ':['HTO.AT','Telecommunications',9.2886],
    'ΟΤΟΕΛ':['OTOEL.AT','Consumer Products & Services',1.2166],
    'ΠΑΠ':['PAP.AT','Personal Care, Drug & Grocery Stores',0.1681],
    'ΠΕΙΡ':['TPEIR.AT','Banks',4.5992],
    'ΠΕΤΡΟ':['PETRO.AT','Industrial Goods & Services',0.0904],
    'ΠΛΑΘ':['PLAT.AT','Industrial Goods & Services',0.3566],
    'ΠΛΑΙΣ':['PLAIS.AT','Retail',0.0661],
    'ΠΛΑΚΡ':['PLAKR.AT','Industrial Goods & Services',0.9924],
    'ΠΡΕΜΙΑ':['PREMIA.AT','Real Estate',0.1121],
    'ΠΡΟΦ':['PROF.AT','Technology',0.2115],
    'ΣΑΡ':['SAR.AT','Personal Care, Drug & Grocery Stores',1.2285],
    'ΣΠΕΙΣ':['SPACE.AT','Telecommunications',0.0761],
    'ΤΕΝΕΡΓ':['TENERGY.AT','Utilities',5.2454],
    'ΦΡΛΚ':['FOYRK.AT','Retail',0.636]
    }

# the column names corresponding to the above dictionary - to be used in the data frame that will be created
COLNAMES = ['YF_ticker', 'industry', 'weight']

In [4]:
# create data frame from dictionary
masterDf = pd.DataFrame.from_dict(ATGSTONKS, orient ='index', columns = COLNAMES)

# create empty dataframe to store prices from Yahoo Finance
pricesDf = pd.DataFrame()

# loop through 'YF_ticker' column
for i in masterDf['YF_ticker']:
    print('fetching prices for',i)
    # get current price and previous close price and store them in the prices data frame
    tickerdata = yf.Ticker(i)
    pricesDf = pricesDf.append({'YF_ticker' : i, 'currentPrice' : tickerdata.info['currentPrice'], 'previousClose' : tickerdata.info['previousClose']}, ignore_index = True)
print('done')

# calculate percentage change between currentPrice and previousClose
pricesDf['pctChange'] = ((pricesDf['currentPrice'] - pricesDf['previousClose']) / pricesDf['currentPrice'] * 100).round(2)

# merge pricesDf into master df
masterDf = masterDf.merge(pricesDf, left_on = 'YF_ticker', right_on = 'YF_ticker')

fetching prices for CENER.AT
fetching prices for TITC.AT
fetching prices for AVAX.AT
fetching prices for ADMIE.AT
fetching prices for ALMY.AT
fetching prices for ALPHA.AT
fetching prices for AEGN.AT
fetching prices for VIO.AT
fetching prices for GEKTERNA.AT
fetching prices for PPC.AT
fetching prices for EEE.AT
fetching prices for ELLAKTOR.AT
fetching prices for ELPE.AT
fetching prices for ELSTR.AT
fetching prices for ELTON.AT
fetching prices for ELHA.AT
fetching prices for ENTER.AT
fetching prices for EPSIL.AT
fetching prices for ETE.AT
fetching prices for EYAPS.AT
fetching prices for EYDAP.AT
fetching prices for EUROB.AT
fetching prices for EXAE.AT
fetching prices for IATR.AT
fetching prices for IKTIN.AT
fetching prices for INKAT.AT
fetching prices for INLIF.AT
fetching prices for INTEK.AT
fetching prices for INTERCO.AT
fetching prices for INTRK.AT
fetching prices for KAMP.AT
fetching prices for KEKR.AT
fetching prices for QUEST.AT
fetching prices for KRI.AT
fetching prices for LAMDA.

## Generate Plotly treemap
We use plotly express: https://plotly.com/python/treemaps/. Note that to get the values displayed correctly we first sort the data frame by `'YF_ticker','industry','pctChange'`.

The color scale can be changed with another one that plotly makes available, run the following to print an image of the available diverging scales:

`fig = px.colors.diverging.swatches_continuous()`

`fig.show()`

Alternatively a custom scale can be used (commented lines 8 - 10)

In [8]:
test = masterDf.sort_values(by=['YF_ticker','industry','pctChange'])

fig = px.treemap(masterDf, 
                 path=[px.Constant('Athens General Composite'), 'industry', 'YF_ticker'], 
                 values = 'weight', #weight
                 color = 'pctChange', #pctChange
                 names = 'YF_ticker',
                 #color_continuous_scale = [ 'Red', "#8b0000", 'Green'],
                 #color_continuous_scale=['#FF0000', "#000000", '#00FF00'],
                 #color_continuous_midpoint=0
                 color_continuous_scale=px.colors.diverging.RdYlGn,
                 color_continuous_midpoint=0,
                 )

fig.data[0].customdata = test['pctChange']
fig.data[0].texttemplate = "%{label}<br>%{customdata}%"

fig.update_traces(textposition = "middle center",  selector = dict(type = 'treemap'))

# Hide the color scale from the chart
fig.update(layout_coloraxis_showscale=False)

# Set theme, margins, tiitle and annotations
fig.update_layout(
    template = "plotly_dark",
    width = 1800,
    height = 800,
    margin = dict(t=100, l=10, r=10, b=70),
    autosize = True,
        title = dict(
        {'text': f'<b>Athens General Composite snapshot</b><br>Rectangle size represents stock weight in the index.<br><sup>last update: {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (15 min delayed data) </sup>',
         'y':0.95,
         'x':0.005},
        font=dict(
            family="Arial",
            size=22)
        ),
        annotations=[
            dict(
                text = "<b>Data source:</b> Yahoo! Finance",
                 showarrow = False,
                 xref = "paper",
                 yref = "paper",
                 x=0,
                 y=-0.05),
            dict(
                text = "<b>Orestis FOTIADIS</b>",
                 showarrow = False,
                 xref = "paper",
                 yref = "paper",
                 x = 0,
                 y = -0.08)
            ]
            )

fig.show()